학습

In [ ]:
import json
import re
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import dump
from lightgbm import LGBMRegressor

warnings.filterwarnings("ignore")

# ==================== 설정 / 하이퍼파라미터 ====================

MODEL_DIR = Path("./models_new_arch")
GLOBAL_MODEL_PATH = MODEL_DIR / "global_new_arch.txt"
META_PATH = MODEL_DIR / "meta_new_arch.json"

# 공행성 탐색
MAX_LAG = 6
MIN_NONZERO = 10       # 한 시계열에서 0이 아닌 월 개수 최소
MIN_LEN = 24           # 상관 계산에 사용할 최소 길이
RECENT_WINDOW = 12     # 최근 상관 계산 창 크기(개월)

CORR_FULL_MIN = 0.20   # 전체 구간 상관 최소 (완화)
SCORE_MIN = 0.0        # 복합 점수 최소 (완화)
TOPK_PER_B = None      # B당 상위 K만 유지하지 않고, 조건 만족 쌍을 모두 사용

# Δlog 클램프 및 naive 블렌딩
CLAMP_Q = 0.80         # |Δlog| 분위수
ALPHA_NAIVE = 0.20     # 최종: (1-α)*model + α*naive

# 전역 LightGBM 하이퍼파라미터
LGBM_PARAMS = dict(
    n_estimators=900,
    learning_rate=0.02,
    max_depth=-1,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
)


# ==================== 유틸 ====================

def _find_file(fname: str) -> Path:
    p = Path(fname)
    if p.exists():
        return p
    p2 = Path("/mnt/data") / fname
    if p2.exists():
        return p2
    raise FileNotFoundError(fname)


def _safe_id(s) -> str:
    return re.sub(r"[^A-Za-z0-9_-]", "_", str(s))[:64]


def safe_corr(x: np.ndarray, y: np.ndarray) -> float:
    if x.size == 0 or y.size == 0:
        return 0.0
    sx = float(np.std(x))
    sy = float(np.std(y))
    if sx == 0.0 or sy == 0.0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])


def load_monthly_and_hs4(train_csv: str):
    """
    monthly: item_id, year, month, hs4, value(합산), ym
    pivot_train: item_id × ym (<= 2025-07)
    item_to_hs4: {item_id(str): hs4(str)}
    hs4_month_logmean: "hs4|month" -> mean log1p(value)
    hs4_global_logmean: hs4 -> mean log1p(value)
    global_logmean: 전체 mean log1p(value)
    """
    p = _find_file(train_csv)
    df = pd.read_csv(
        p,
        dtype={
            "item_id": str,
            "year": int,
            "month": int,
            "hs4": str,
            "value": float,
        },
    )

    monthly = (
        df.groupby(["item_id", "year", "month"], as_index=False)
        .agg({"value": "sum", "hs4": "first"})
    )
    monthly["ym"] = pd.to_datetime(
        monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
    )

    pivot = (
        monthly.pivot(index="item_id", columns="ym", values="value")
        .fillna(0.0)
        .sort_index(axis=1)
    )
    pivot_train = pivot.loc[:, pivot.columns <= pd.Timestamp("2025-07-01")]

    def _mode_or_first(s):
        m = s.mode()
        return m.iloc[0] if not m.empty else s.iloc[0]

    item_hs4_s = monthly.groupby("item_id")["hs4"].agg(_mode_or_first)
    item_to_hs4 = item_hs4_s.to_dict()

    monthly["logv"] = np.log1p(monthly["value"])
    grp_hm = monthly.groupby(["hs4", "month"])["logv"].mean()
    hs4_month_logmean = {f"{h}|{m}": float(v) for (h, m), v in grp_hm.items()}

    grp_h = monthly.groupby("hs4")["logv"].mean()
    hs4_global_logmean = {h: float(v) for h, v in grp_h.items()}

    global_logmean = float(monthly["logv"].mean())

    return pivot_train, item_to_hs4, hs4_month_logmean, hs4_global_logmean, global_logmean


# ==================== 1) 공행성쌍 탐색 (복합 점수, 완화된 선택) ====================

def find_comovement_pairs_multi(
    pivot_val: pd.DataFrame,
    max_lag: int,
    min_nonzero: int,
    min_len: int,
    recent_window: int,
    corr_full_min: float,
    score_min: float,
    topk_per_B=None,
) -> pd.DataFrame:
    """
    각 (A,B)에 대해:
      - lag 1..max_lag 로 log1p 기준 cross-corr
      - 전체 상관, 최근 상관, 전/후반기 안정성으로 점수 계산
      - corr_full_min, score_min 조건을 만족하는 쌍 모두 사용
        (topk_per_B가 None이 아니면 B별 상위 K만 유지)
    """
    items = pivot_val.index.to_list()
    dates = pivot_val.columns.to_list()
    T = len(dates)
    pivot_log = np.log1p(pivot_val)

    rows = []

    for A in tqdm(items, desc="scan leaders (multi-criteria)"):
        x_val = pivot_val.loc[A].values.astype(float)
        if np.count_nonzero(x_val) < min_nonzero:
            continue
        x_log = pivot_log.loc[A].values.astype(float)

        for B in items:
            if B == A:
                continue
            y_val = pivot_val.loc[B].values.astype(float)
            if np.count_nonzero(y_val) < min_nonzero:
                continue
            y_log = pivot_log.loc[B].values.astype(float)

            best = None

            for lag in range(1, max_lag + 1):
                if T - lag < min_len:
                    continue

                x = x_log[:-lag]
                y = y_log[lag:]
                L = len(x)
                if L < min_len:
                    continue

                corr_full = safe_corr(x, y)

                half = L // 2
                if half >= 2 and (L - half) >= 2:
                    corr1 = safe_corr(x[:half], y[:half])
                    corr2 = safe_corr(x[half:], y[half:])
                else:
                    corr1 = corr2 = corr_full
                stability = abs(corr1 - corr2)

                rw = min(recent_window, L)
                xr = x[-rw:]
                yr = y[-rw:]
                corr_recent = safe_corr(xr, yr)

                score = (
                    abs(corr_full)
                    + 0.5 * abs(corr_recent)
                    - 0.3 * stability
                )

                if (best is None) or (score > best["score"]):
                    best = {
                        "lag": lag,
                        "corr_full": corr_full,
                        "corr_recent": corr_recent,
                        "stability": stability,
                        "score": score,
                    }

            if best is None:
                continue

            rows.append(
                {
                    "leading_item_id": A,
                    "following_item_id": B,
                    "best_lag": int(best["lag"]),
                    "corr_full": float(best["corr_full"]),
                    "corr_recent": float(best["corr_recent"]),
                    "stability": float(best["stability"]),
                    "score": float(best["score"]),
                }
            )

    df_all = pd.DataFrame(rows)
    if df_all.empty:
        return df_all

    df_all["corr_full_abs"] = df_all["corr_full"].abs()

    # 조건 필터
    cond = (df_all["corr_full_abs"] >= corr_full_min) & (df_all["score"] >= score_min)
    df = df_all[cond]

    # topk_per_B가 지정되면 B별 상위 K만 유지 (지금은 None, 즉 전체 사용)
    if topk_per_B is not None:
        selected = []
        for B, grp in df.groupby("following_item_id"):
            grp_sorted = grp.sort_values("score", ascending=False)
            selected.append(grp_sorted.head(topk_per_B))
        df = pd.concat(selected, ignore_index=True)
    else:
        df = df.reset_index(drop=True)

    return df


# ==================== 2) 전역 logB_{t+1} LightGBM 학습 ====================

def build_training_data_global(
    pivot: pd.DataFrame,
    pairs: pd.DataFrame,
    item_to_hs4: dict,
    hs4_month_logmean: dict,
    hs4_global_logmean: dict,
    global_logmean: float,
) -> pd.DataFrame:
    """
    전역 LightGBM 학습용 데이터.

    X:
      - logB_t, logB_t_1, logB_t_2
      - logA_t_lag, logA_t_lag_1
      - corr_full, corr_recent, stability, score, best_lag
      - sin_m, cos_m (다음달)
      - hs4A_mlog_curr (A, 현재월), hs4B_mlog_next (B, 다음달)
    y:
      - log1p(B_{t+1})
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for r in pairs.itertuples(index=False):
        A = r.leading_item_id
        B = r.following_item_id
        lag = int(r.best_lag)
        corr_full = float(r.corr_full)
        corr_recent = float(r.corr_recent)
        stability = float(r.stability)
        score = float(r.score)

        if (A not in pivot.index) or (B not in pivot.index):
            continue

        sA = pivot.loc[A].values.astype(float)
        sB = pivot.loc[B].values.astype(float)

        hs4_A = item_to_hs4.get(A, None)
        hs4_B = item_to_hs4.get(B, None)

        for t in range(max(lag, 2), n_months - 1):
            B_t = sB[t]
            B_t_1 = sB[t - 1]
            B_t_2 = sB[t - 2]
            B_t1 = sB[t + 1]

            A_t_lag = sA[t - lag]
            if (t - lag - 1) >= 0:
                A_t_lag_1 = sA[t - lag - 1]
            else:
                A_t_lag_1 = A_t_lag

            logB_t = np.log1p(B_t)
            logB_t_1 = np.log1p(B_t_1)
            logB_t_2 = np.log1p(B_t_2)
            logB_t1 = np.log1p(B_t1)
            logA_t_lag = np.log1p(A_t_lag)
            logA_t_lag_1 = np.log1p(A_t_lag_1)

            m_curr = months[t].month
            m_next = months[t + 1].month
            sin_m = np.sin(2 * np.pi * m_next / 12.0)
            cos_m = np.cos(2 * np.pi * m_next / 12.0)

            if hs4_A is not None:
                key_A_curr = f"{hs4_A}|{m_curr}"
                mlog_A_curr = hs4_month_logmean.get(
                    key_A_curr,
                    hs4_global_logmean.get(hs4_A, global_logmean),
                )
            else:
                mlog_A_curr = global_logmean

            if hs4_B is not None:
                key_B_next = f"{hs4_B}|{m_next}"
                mlog_B_next = hs4_month_logmean.get(
                    key_B_next,
                    hs4_global_logmean.get(hs4_B, global_logmean),
                )
            else:
                mlog_B_next = global_logmean

            rows.append(
                {
                    "logB_t": logB_t,
                    "logB_t_1": logB_t_1,
                    "logB_t_2": logB_t_2,
                    "logA_t_lag": logA_t_lag,
                    "logA_t_lag_1": logA_t_lag_1,
                    "corr_full": corr_full,
                    "corr_recent": corr_recent,
                    "stability": stability,
                    "score": score,
                    "best_lag": float(lag),
                    "sin_m": sin_m,
                    "cos_m": cos_m,
                    "hs4A_mlog_curr": float(mlog_A_curr),
                    "hs4B_mlog_next": float(mlog_B_next),
                    "target_logB_next": logB_t1,
                    "B": B,
                }
            )

    return pd.DataFrame(rows)


# ==================== 메인 학습 ====================

def main():
    MODEL_DIR.mkdir(parents=True, exist_ok=True)

    pivot_train, item_to_hs4, hs4_month_logmean, hs4_global_logmean, global_logmean = \
        load_monthly_and_hs4("train.csv")

    pivot_log = np.log1p(pivot_train)

    # 1) 공행성쌍 (완화된 multi-criteria)
    pairs = find_comovement_pairs_multi(
        pivot_val=pivot_train,
        max_lag=MAX_LAG,
        min_nonzero=MIN_NONZERO,
        min_len=MIN_LEN,
        recent_window=RECENT_WINDOW,
        corr_full_min=CORR_FULL_MIN,
        score_min=SCORE_MIN,
        topk_per_B=TOPK_PER_B,
    )
    print("pairs (multi-criteria, relaxed):", len(pairs))

    # 2) 전역 LightGBM 학습
    df_train = build_training_data_global(
        pivot_train,
        pairs,
        item_to_hs4,
        hs4_month_logmean,
        hs4_global_logmean,
        global_logmean,
    )
    if df_train.empty:
        meta = {
            "pairs": [],
            "global_feature_cols": [],
            "clamp_q": CLAMP_Q,
            "alpha_naive": ALPHA_NAIVE,
            "clamp_by_B": {},
            "item_to_hs4": item_to_hs4,
            "hs4_month_logmean": hs4_month_logmean,
            "hs4_global_logmean": hs4_global_logmean,
            "global_logmean": global_logmean,
            "hp": {
                "MAX_LAG": MAX_LAG,
                "MIN_NONZERO": MIN_NONZERO,
                "MIN_LEN": MIN_LEN,
                "RECENT_WINDOW": RECENT_WINDOW,
                "CORR_FULL_MIN": CORR_FULL_MIN,
                "SCORE_MIN": SCORE_MIN,
                "TOPK_PER_B": TOPK_PER_B,
            },
        }
        with open(META_PATH, "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)
        print("No training rows. Saved empty meta.")
        return

    feature_cols = [
        "logB_t",
        "logB_t_1",
        "logB_t_2",
        "logA_t_lag",
        "logA_t_lag_1",
        "corr_full",
        "corr_recent",
        "stability",
        "score",
        "best_lag",
        "sin_m",
        "cos_m",
        "hs4A_mlog_curr",
        "hs4B_mlog_next",
    ]
    X = df_train[feature_cols].values
    y = df_train["target_logB_next"].values

    lgbm = LGBMRegressor(**LGBM_PARAMS)
    lgbm.fit(X, y)
    lgbm.booster_.save_model(str(GLOBAL_MODEL_PATH))

    # 3) follower별 Δlog 분포로 클램프 값 계산
    clamp_by_B = {}
    for B in pivot_train.index:
        sB = pivot_train.loc[B].values.astype(float)
        logs = np.log1p(sB)
        d = np.diff(logs)
        if len(d) == 0:
            c = 0.0
        else:
            c = float(np.quantile(np.abs(d), CLAMP_Q))
        if c < 0.05:
            c = 0.05
        clamp_by_B[B] = c

    # 4) 메타 저장
    meta = {
        "pairs": pairs.to_dict(orient="records"),
        "global_feature_cols": feature_cols,
        "clamp_q": CLAMP_Q,
        "alpha_naive": ALPHA_NAIVE,
        "clamp_by_B": clamp_by_B,
        "item_to_hs4": item_to_hs4,
        "hs4_month_logmean": hs4_month_logmean,
        "hs4_global_logmean": hs4_global_logmean,
        "global_logmean": global_logmean,
        "hp": {
            "MAX_LAG": MAX_LAG,
            "MIN_NONZERO": MIN_NONZERO,
            "MIN_LEN": MIN_LEN,
            "RECENT_WINDOW": RECENT_WINDOW,
            "CORR_FULL_MIN": CORR_FULL_MIN,
            "SCORE_MIN": SCORE_MIN,
            "TOPK_PER_B": TOPK_PER_B,
        },
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"train_rows={len(df_train)} , pairs={len(pairs)}")
    print(f"saved: {GLOBAL_MODEL_PATH} , {META_PATH}")


if __name__ == "__main__":
    main()

scan leaders (multi-criteria): 100%|██████████| 100/100 [00:25<00:00,  3.96it/s]


pairs (multi-criteria, relaxed): 5427
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2833
[LightGBM] [Info] Number of data points in the train set: 207558, number of used features: 14
[LightGBM] [Info] Start training from score 12.076956
train_rows=207558 , pairs=5427
saved: models_new_arch/global_new_arch.txt , models_new_arch/meta_new_arch.json


추론

In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from lightgbm import Booster

MODEL_DIR = Path("./models_new_arch")
GLOBAL_MODEL_PATH = MODEL_DIR / "global_new_arch.txt"
META_PATH = MODEL_DIR / "meta_new_arch.json"


def _find_file(fname: str) -> Path:
    p = Path(fname)
    if p.exists():
        return p
    p2 = Path("/mnt/data") / fname
    if p2.exists():
        return p2
    raise FileNotFoundError(fname)


def load_pivot_full(train_csv: str) -> pd.DataFrame:
    """
    item_id × ym 피벗 (2025-08까지 열 보강).
    index: item_id(str), columns: 월(1일 Timestamp)
    """
    p = _find_file(train_csv)
    df = pd.read_csv(
        p,
        dtype={"item_id": str, "year": int, "month": int, "value": float},
    )
    monthly = df.groupby(["item_id", "year", "month"], as_index=False)["value"].sum()
    monthly["ym"] = pd.to_datetime(
        monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
    )
    start = monthly["ym"].min()
    end = pd.Timestamp("2025-08-01")
    idx = pd.date_range(start=start, end=end, freq="MS")
    items = np.sort(monthly["item_id"].unique())

    pivot = pd.DataFrame(0.0, index=items, columns=idx, dtype=float)
    for iid, g in monthly.groupby("item_id"):
        s = g.set_index("ym")["value"].reindex(idx, fill_value=0.0).astype(float)
        pivot.loc[iid, :] = s.values

    pivot = pivot.sort_index(axis=1)
    return pivot


def main():
    with open(META_PATH, "r", encoding="utf-8") as f:
        meta = json.load(f)

    pairs = pd.DataFrame(meta.get("pairs", []))
    if pairs.empty:
        sub = pd.DataFrame(
            columns=["leading_item_id", "following_item_id", "value"]
        )
        sub.to_csv("submission_new_arch.csv", index=False)
        print("No pairs in meta. Wrote empty submission_new_arch.csv")
        return

    feature_cols = meta.get("global_feature_cols", [])
    alpha = float(meta.get("alpha_naive", 0.20))
    clamp_by_B = meta.get("clamp_by_B", {})

    item_to_hs4 = meta.get("item_to_hs4", {})
    hs4_month_logmean = meta.get("hs4_month_logmean", {})
    hs4_global_logmean = meta.get("hs4_global_logmean", {})
    global_logmean = float(meta.get("global_logmean", 0.0))

    global_model = Booster(model_file=str(GLOBAL_MODEL_PATH))

    pivot_full = load_pivot_full("train.csv")
    months = pivot_full.columns.to_list()
    assert months[-1] == pd.Timestamp("2025-08-01"), "마지막 열은 2025-08-01이어야 합니다."
    t_last_train = len(months) - 2  # 2025-07 index

    out_rows = []

    for r in pairs.itertuples(index=False):
        A = r.leading_item_id
        B = r.following_item_id
        lag = int(r.best_lag)
        corr_full = float(r.corr_full)
        corr_recent = float(r.corr_recent)
        stability = float(r.stability)
        score = float(r.score)

        if (A not in pivot_full.index) or (B not in pivot_full.index):
            continue

        sA = pivot_full.loc[A].values.astype(float)
        sB = pivot_full.loc[B].values.astype(float)

        B_t = sB[t_last_train]
        B_t_1 = sB[t_last_train - 1] if t_last_train - 1 >= 0 else 0.0
        B_t_2 = sB[t_last_train - 2] if t_last_train - 2 >= 0 else 0.0

        if (t_last_train - lag) >= 0:
            A_t_lag = sA[t_last_train - lag]
        else:
            A_t_lag = 0.0

        if (t_last_train - lag - 1) >= 0:
            A_t_lag_1 = sA[t_last_train - lag - 1]
        else:
            A_t_lag_1 = A_t_lag

        logB_t = np.log1p(B_t)
        logB_t_1 = np.log1p(B_t_1)
        logB_t_2 = np.log1p(B_t_2)
        logA_t_lag = np.log1p(A_t_lag)
        logA_t_lag_1 = np.log1p(A_t_lag_1)

        m_curr = months[t_last_train].month   # 2025-07
        m_next = months[t_last_train + 1].month  # 2025-08
        sin_m = np.sin(2 * np.pi * m_next / 12.0)
        cos_m = np.cos(2 * np.pi * m_next / 12.0)

        hs4_A = item_to_hs4.get(A, None)
        hs4_B = item_to_hs4.get(B, None)

        if hs4_A is not None:
            key_A_curr = f"{hs4_A}|{m_curr}"
            mlog_A_curr = hs4_month_logmean.get(
                key_A_curr,
                hs4_global_logmean.get(hs4_A, global_logmean),
            )
        else:
            mlog_A_curr = global_logmean

        if hs4_B is not None:
            key_B_next = f"{hs4_B}|{m_next}"
            mlog_B_next = hs4_month_logmean.get(
                key_B_next,
                hs4_global_logmean.get(hs4_B, global_logmean),
            )
        else:
            mlog_B_next = global_logmean

        feat_dict = {
            "logB_t": logB_t,
            "logB_t_1": logB_t_1,
            "logB_t_2": logB_t_2,
            "logA_t_lag": logA_t_lag,
            "logA_t_lag_1": logA_t_lag_1,
            "corr_full": corr_full,
            "corr_recent": corr_recent,
            "stability": stability,
            "score": score,
            "best_lag": float(lag),
            "sin_m": sin_m,
            "cos_m": cos_m,
            "hs4A_mlog_curr": float(mlog_A_curr),
            "hs4B_mlog_next": float(mlog_B_next),
        }

        X = np.array([[feat_dict[col] for col in feature_cols]], dtype=float)
        log_pred = float(global_model.predict(X)[0])

        d_log_pred = log_pred - logB_t
        cB = float(clamp_by_B.get(B, 0.1))
        d_log_clamped = float(np.clip(d_log_pred, -cB, cB))
        log_pred_clamped = logB_t + d_log_clamped

        y_pred = float(np.expm1(log_pred_clamped))
        y_final = (1.0 - alpha) * y_pred + alpha * B_t
        y_final = max(0.0, y_final)

        out_rows.append(
            {
                "leading_item_id": A,
                "following_item_id": B,
                "value": int(np.rint(y_final)),
            }
        )

    sub = pd.DataFrame(
        out_rows, columns=["leading_item_id", "following_item_id", "value"]
    )
    sub.to_csv("submission_new_arch.csv", index=False)
    print(f"Wrote submission_new_arch.csv with {len(sub)} rows")


if __name__ == "__main__":
    main()

Wrote submission_new_arch.csv with 5427 rows
